# Development Notebook

This notebook is made for experimenting purpose, in order to make new mashup technics you can add in the automashup-app/mashup.py file. 

The aim is to create new mashup technics within the /automashup-app/mashup.py file. Then, if you use the same model as the other functions, it's gonna be easy to add it in the app

## Example of a working mashup Method

The following cells are an example of a mashup method. You may copy and paste the following example at the end of the file to make your own method. Here we assume the tracks are already separated and analyzed so you just have to run the following cell, not to modify it (unless you want to try with other songs for instance).

## Preprocess a song 

Before applying our mashup methods, we'll be getting some data out of the song we want to use.

For this notebook to be interesting, we suggest you to preprocess two songs first

In [ ]:
import allin1
import os
from IPython.display import Audio

os.chdir("../automashup-app/")

from utils import key_finder, increase_array_size
from track import Track

os.chdir("../automashup-notebook/")

In [ ]:
path = "" # Path to the song
allin1.analyze(path, out_dir='./struct', demix_dir='./separated', keep_byproducts=True, overwrite=True)
key_finder(path)

### Loading the preprocessed songs as input for mashup

Once you preprocessed a song, it will be identified by a song name, which is the name of the file preprocessed, without the extension. For instance, for the song "./input/song.mp3", the track name is "song"

In [ ]:
# Put your song names here
song_name_1 = ''
song_name_2 = ''
# You will always have at least one track and up to four
# If you need one track as a reference (for instance for beat structure), you will use the first one

What we'll be calling a "track" is a python Track object with the following attributes : 
  
 
* 'track_name' : String, 
* 'audio' : audio of the track, it's a np array it can be only a part of the song (vocals, instru, ...),
* 'sr' : the sampling frequency, 
* 'path' : the path of the original file
* 'bpm' : the bpm found by allin1 analysis
* 'beats' : beats list determined by allin1 analysis
* 'downbeats' : downbeats list determined by allin1 analysis
* 'segments' : list of the song's phases with their phase label (verse, chorus, ...)
* 'key' : correlation with each key (the highest value will be used as main key)
     

It's important that **your mashup methods returns the same kind of object !** Also, your method should handle up to 4 different tracks !
Especially, metadata should look like this :

{
  "path": "/home/gaubiche/Documents/MCE/Automashup/AutoMashup/mashup/input/bazard\u00e9e.mp3",
  "bpm": 103,
  "beats": [
    4.06,
    4.63,
    5.23,
    5.81,
    ...
  ],
  "downbeats": [
    5.23,
    7.57,
    9.89,
    12.23,
    14.56,
    16.88,
    19.21,
    21.54,
    ...
  ],
  "beat_positions": [
    3,
    4,
    1,
    2,
    3,
    4,
    1,
    ...
  ],
  "segments": [
    {
      "start": 0.0,
      "end": 4.06,
      "label": "start"
    },
    {
      "start": 4.06,
      "end": 32.59,
      "label": "verse"
    },
    {
      "start": 32.59,
      "end": 51.21,
      "label": "chorus"
    },
    ...
  ],
  "key": {
    "C major": -0.512,
    "C# major": 0.29,
    ...
  }
}

Do not freak out ! Metadatas are created within the analysis (within each track) so you won't have to set everything by hand. However, try to modify the metadatas according to the modification you do to a track to keep them consistent along the process !

In [ ]:
## Let's create some Track objects with our preprocessed songs
# We'll be loading the vocals of the first song and the whole instru 
# of the second song

tracks =  [] # input of the mashup methods

# type attribute enables to choose a separated part of a song (from demucs source separation)
# it can be 'vocals', 'bass', 'drums' or 'other'
track_1 = Track.track_from_song(song_name_1, type='vocals')
track_2 = Track.track_from_song(song_name_2, type='bass')
track_3 = Track.track_from_song(song_name_2, type='drums')
track_4 = Track.track_from_song(song_name_2, type='other')

tracks = [track_1, track_2, track_3, track_4]

In [ ]:
# We can have access to some attributes : 
print(f"BPM : {track_1.bpm}")
print(f"Key correlation : {track_1.key}")
print(f"Beat frames : {track_1.beats}")
print(f"Track audio : {track_1.audio}")
print(f"Track Sampling Frequency {track_1.sr}")

In [ ]:
# We have also some built-in functions for Track objects : 
print(f"Key calculation (key with highest value in key correlation) : {track_1.get_key()}")

Audio(track_1.audio, rate = track_1.sr)

# To repitch to another key : 
track_1.pitch_track("C major")
Audio(track_1.audio, rate = track_1.sr)

Audio(track_1.audio, rate = track_1.sr)
# To align one track phases (chorus, verse, ...) to another one's
track_2.fit_phase(track_1)
Audio(track_2.audio, rate = track_2.sr)

# To add a metronome sound to audio
track_1.add_metronome()
Audio(track_1.audio, rate = track_1.sr)

# lets reload our tracks from scratch to cancel our changes
track_1 = Track.track_from_song(song_name_1, type='vocals')
track_2 = Track.track_from_song(song_name_2, type='bass')

tracks = [track_1, track_2, track_3, track_4]

### Mashup Example

In [ ]:
os.chdir("../automashup-app/") # In order to reach the file we want to load

#### YOUR METHOD HERE :
### You have to write your method in "../automashup-app/mashup.py"

import mashup

import importlib
importlib.reload(mashup)

os.chdir("../automashup-notebook/") # In order to get back to our directory

### Apply the method here

mashup = mashup.mashup_technic(tracks)

In [ ]:
# Have a listen
Audio(mashup.audio, rate = mashup.sr)

In [ ]:
import soundfile as sf
### Save the file : 
sf.write("mashup.wav", mashup.audio, mashup.sr)

The code of the mashup method is the following (you can also find it in the mashup.py file) : 

In [ ]:
def mashup_technic(tracks):
    # Mashup technic with first downbeat alignment and bpm sync
    sr = tracks[0].sr # The first track is used to determine the target bpm
    tempo = tracks[0].bpm
    beginning_instant = tracks[0].downbeats[0] # downbeats metadata
    beginning = beginning_instant * sr
    mashup = np.zeros(0)
    mashup_name = ""

    # we add each track to the mashup
    for track in tracks:
        mashup_name += track.name + " " # name
        track_tempo = track.bpm
        track_beginning_temporal = track.downbeats[0]
        track_sr = track.sr
        track_beginning = track_beginning_temporal * track_sr
        track_audio = track.audio

        # reset first downbeat position
        track_audio_no_offset = np.array(track_audio)[round(track_beginning):] 

        # multiply by bpm rate
        track_audio_accelerated = librosa.effects.time_stretch(track_audio_no_offset, rate = tempo / track_tempo)

        # add the right number of zeros to align with the main track
        final_track_audio = np.concatenate((np.zeros(round(beginning)), track_audio_accelerated)) 

        size = max(len(mashup), len(final_track_audio))
        mashup = np.array(mashup)
        mashup = (increase_array_size(final_track_audio, size) + increase_array_size(mashup, size))

    # we return a modified version of the first track
    # doing so, we keep its metadata
    tracks[0].audio = mashup

    return tracks[0]

Now try to make your own !